In [25]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

print("Загружаем датасет...")
dataset, info = tfds.load('tiny_shakespeare', with_info=True, as_supervised=False)

Загружаем датасет...


In [26]:
# Получаем текст из датасета
train_data = dataset['train']
text = ""

# Объеденяем все в один текст
for example in train_data:
    text += example['text'].numpy().decode('utf-8')

print(f"Общая длинна текста: {len(text)} символов")
print(f"Первые 100 символов: {text[:100]}")

Общая длинна текста: 1003854 символов
Первые 100 символов: First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [27]:
print("Создаем словарь...")
vocab = sorted(set(text))
vocab_size = len(vocab)
print(f"Размер словаря: {vocab_size} символов")

# Создаем маппин индекс=>символ
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = np.array(vocab)

# Конвертируем текст в индексы
text_as_int = np.array([char2idx[char] for char in text])

# Создаем датасет для обучения
seq_length = 100  # Длинна текста

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# генерируем текст данной длинны
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

Создаем словарь...
Размер словаря: 65 символов


In [28]:
def split_input_target(chunk):
    """Разделям на параметр и результат"""
    input_text = chunk[:-1]  # Все токены кроме последнего
    target_text = chunk[1:]   # Всте токены кроме первого
    return input_text, target_text

# разделяем для каждого предложения
dataset = sequences.map(split_input_target)

In [29]:
# Количество для обучения
BATCH_SIZE = 1
BUFFER_SIZE = 10000

# Перемешиваем датасет
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Строим модель
def build_model(vocab_size, embedding_dim=256, rnn_units=1024):
    # Входной слой
    inputs = tf.keras.layers.Input(shape=(None,), batch_size=BATCH_SIZE)

    # Embedding слой
    x = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(inputs)

    # Первый LSTM слой
    x = LSTM(
        rnn_units,
        return_sequences=True,
        stateful=True,
        recurrent_initializer='glorot_uniform'
    )(x)
    x = Dropout(0.2)(x)

    # Второй LSTM слой
    x = LSTM(
        rnn_units,
        return_sequences=True,
        stateful=True,
        recurrent_initializer='glorot_uniform'
    )(x)
    x = Dropout(0.2)(x)

    # Выходной ряд
    outputs = Dense(vocab_size)(x)

    # Создаем модель
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

model = build_model(vocab_size=vocab_size)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (1, None)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_7 (Embedding)         │ (1, None, 256)         │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (1, None, 1024)        │     5,246,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (1, None, 1024)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (1, None, 1024)        │     8,392,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (1, None, 1024)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (1, None, 65)          │        66,625 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,722,945 (52.35 MB)

 Trainable params: 13,722,945 (52.35 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Компилируем модель
def loss(labels, logits):
    """кастомная функция потери"""
    return tf.keras.losses.sparse_categorical_crossentropy(
        labels, logits, from_logits=True
    )

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=loss
)

early_stopping = EarlyStopping(
    monitor='loss',
    patience=5,
    restore_best_weights=True
)

# Обучаем модель
EPOCHS = 30

history = model.fit(
    dataset,
    epochs=EPOCHS,
    callbacks=[early_stopping]
)

Epoch 1/30
 405/9939 ━━━━━━━━━━━━━━━━━━━━ 8:57:48 3s/step - loss: 3.1113

In [31]:
# Генерация текста
def generate_text(model, start_string, num_generate=1000, temperature=1.0):
    """Генерируем текст"""
    # Конвертируем строку в токены
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Пустая строка, где храним результат
    text_generated = []


    for i in range(num_generate):
        predictions = model(input_eval)
        # Убираем одномерные размерности
        predictions = tf.squeeze(predictions, 0)

        # Предсказываем следующий токен
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Передаем предсказаный токен в модель
        input_eval = tf.expand_dims([predicted_id], 0)

        # Добавляем результат в сгенерированную строгу
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

print("\nГенерируем тестовый текст...")
generated_text = generate_text(model, start_string="ROMEO: ", num_generate=200)
print(generated_text)



Генерируем тестовый текст...
ROMEO: fyhonwthhdoratirFteiekqDillgaixhwMfm?'cioiOV,Yg
-
,t-iotVBuuhr:!eatrtrJetpodHeochM-ec?oL ikVu;oqeOMkvYhmohweiu?Tg,nxmWiwbtSRwwlIewrAoFzMbyiiAxDoFImenrf&o. Aoi!Le!Tnlzstshsijq-y'itRi,T
UIrTmUU

DDashNs
